In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from IPython.display import Image
# from jupyterthemes import jtplot
from matplotlib.colors import ListedColormap
import mglearn
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import classification_report
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import GroupShuffleSplit
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc, roc_auc_score
from scipy import interp
from math import sqrt
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neural_network import MLPClassifier

In [ ]:
data = pd.read_csv('cleaned_data1.csv')
data.columns

In [ ]:
dropvar = ['assigned_room_type.P', 'reserved_room_type.P']
data = data.drop(dropvar, axis = 1)
data

## Ridge Logisitic Regression

In [ ]:
dropvar = ['arrival_date_month.December','arrival_date_month.February', 'Market.Direct', 'Market.Groups', 
           'Distribution.TA_TO', 'arrival_date_month.October', 'arrival_date_month.April', 'arrival_date_month.November', 'Meal.BB', 
           'assigned_room_type.L']
data_ridge = data.drop(dropvar, axis = 1)
data_ridge

In [ ]:
y = data_ridge[['is_canceled']]
x = data_ridge.drop(['is_canceled'], axis = 1)
x

In [ ]:
#logisitic regression ridge
log_ridge = LogisticRegression()

# Create regularization penalty space
penalty = ['l1', 'l2']

# Create regularization hyperparameter space
C = [0.001,.009,0.01,.09,1,2,3,4,5,8,9,10,25]

# Create regularization hyperparameter space
# max_iter = range(100,1000,100)

# Create hyperparameter options
hyperparameters = dict(C=C, penalty=penalty)

In [ ]:
# Create grid search using 5-fold cross validation
log_ridge = GridSearchCV(log_ridge, hyperparameters, cv=10, verbose=0, scoring = 'f1')
# Fit grid search
best_model = log_ridge.fit(x, y)

In [ ]:
# View best hyperparameters
print('Best Penalty:', best_model.best_estimator_.get_params()['penalty'])
print('Best C:', best_model.best_estimator_.get_params()['C'])
# print('Best max_iter:', best_model.best_estimator_.get_params()['max_iter'])

In [8]:
# , max_iter = 1000
log_best_par = LogisticRegression(penalty = 'l2', C = 10, max_iter = 1000)
kappa1 = []
rmse1 = []
f1_1 = []

kf = StratifiedKFold(n_splits=10, random_state=0, shuffle=True)
for train_index, test_index in kf.split(x,y):
    x_train, x_test = x[x.index.isin(train_index)], x[x.index.isin(test_index)]
    y_train, y_test = y[y.index.isin(train_index)], y[y.index.isin(test_index)]
    log_best_par.fit(x_train, y_train)
    log_best_par_pred = log_best_par.predict(x_test)
    kappa = metrics.cohen_kappa_score(y_test, log_best_par_pred)
    kappa1.append(kappa)
    f1 = metrics.f1_score(y_test, log_best_par_pred)
    f1_1.append(f1)
    rmse = sqrt(metrics.mean_squared_error(y_test, log_best_par_pred))
    rmse1.append(rmse)
print('kappa',np.mean(kappa1), 'f1', np.mean(f1_1), 'rmse', np.mean(rmse1))

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

kappa 0.57336467417301 f1 0.7077922191724328 rmse 0.4340270028247348


## Decision Tree

In [ ]:
param_grid = {'max_depth': np.arange(3, 10),
             'min_samples_leaf': np.arange(2,11),
             'min_samples_split': range(2,4,1)}

In [ ]:
tree = DecisionTreeClassifier()

tree = GridSearchCV(tree, param_grid, cv=10, scoring = 'f1')
tree.fit(x,y)

In [ ]:
print('Best max_depth:', tree.best_estimator_.get_params()['max_depth'])
print('Best min_samples_leaf:', tree.best_estimator_.get_params()['min_samples_leaf'])
print('Best min_samples_split:', tree.best_estimator_.get_params()['min_samples_split'])

In [9]:
tree_best_par = DecisionTreeClassifier(max_depth = 6, min_samples_leaf = 10, min_samples_split = 2)
kappa2 = []
rmse2 = []
f1_2 = []

kf = StratifiedKFold(n_splits=10, random_state=0, shuffle=True)
for train_index, test_index in kf.split(x,y):
    x_train, x_test = x[x.index.isin(train_index)], x[x.index.isin(test_index)]
    y_train, y_test = y[y.index.isin(train_index)], y[y.index.isin(test_index)]
    tree_best_par.fit(x_train, y_train)
    tree_best_par_pred = tree_best_par.predict(x_test)
    kappa = metrics.cohen_kappa_score(y_test, tree_best_par_pred)
    kappa2.append(kappa)
    f1 = metrics.f1_score(y_test, tree_best_par_pred)
    f1_2.append(f1)
    rmse = sqrt(metrics.mean_squared_error(y_test, tree_best_par_pred))
    rmse2.append(rmse)
print('kappa',np.mean(kappa2), 'f1', np.mean(f1_2), 'rmse', np.mean(rmse2))

kappa 0.5659423783010221 f1 0.7024588946713007 rmse 0.43767419838190824


## Random Forest

In [10]:
rf_best_par = RandomForestClassifier()
kappa3 = []
rmse3 = []
f1_3 = []

kf = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
for train_index, test_index in kf.split(x,y):
    x_train, x_test = x[x.index.isin(train_index)], x[x.index.isin(test_index)]
    y_train, y_test = y[y.index.isin(train_index)], y[y.index.isin(test_index)]
    rf_best_par.fit(x_train, y_train)
    rf_best_par_pred = rf_best_par.predict(x_test)
    kappa = metrics.cohen_kappa_score(y_test, rf_best_par_pred)
    kappa3.append(kappa)
    f1 = metrics.f1_score(y_test, rf_best_par_pred)
    f1_3.append(f1)
    rmse = sqrt(metrics.mean_squared_error(y_test, rf_best_par_pred))
    rmse3.append(rmse)
print('kappa',np.mean(kappa3), 'f1', np.mean(f1_3), 'rmse', np.mean(rmse3))

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to

kappa 0.6994146419888138 f1 0.804806555953955 rmse 0.37051652383827616


## Find out optimal parameter

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.1, random_state = 0)

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
# max_features = ['auto', 'sqrt','log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = range(2,11)
# Minimum number of samples required at each leaf node
min_samples_leaf = range(2,11)
# Method of selecting samples for training each tree
# bootstrap = [True, False]

# criterion = ['gini', 'entropy']

# Create the random grid
random_grid = {'n_estimators': n_estimators,
#                'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
#                'min_samples_leaf': min_samples_leaf
#                'bootstrap': bootstrap,
#               'criterion': criterion
              }

rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 50, cv = 3, verbose=2, random_state=0, n_jobs = -1)
rf_random.fit(x_train, y_train)

In [ ]:
rf_random.best_params_

In [11]:
rf_best_par_ = RandomForestClassifier(n_estimators = 1800, min_samples_split = 2, max_depth = 140)
kappa4 = []
rmse4 = []
f1_4 = []

kf = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
for train_index, test_index in kf.split(x,y):
    x_train, x_test = x[x.index.isin(train_index)], x[x.index.isin(test_index)]
    y_train, y_test = y[y.index.isin(train_index)], y[y.index.isin(test_index)]
    rf_best_par_.fit(x_train, y_train)
    rf_best_par_pred1 = rf_best_par_.predict(x_test)
    kappa = metrics.cohen_kappa_score(y_test, rf_best_par_pred1)
    kappa4.append(kappa)
    f1 = metrics.f1_score(y_test, rf_best_par_pred1)
    f1_4.append(f1)
    rmse = sqrt(metrics.mean_squared_error(y_test, rf_best_par_pred1))
    rmse4.append(rmse)
print('kappa',np.mean(kappa4), 'f1', np.mean(f1_4), 'rmse', np.mean(rmse4))

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to

kappa 0.7020388234209302 f1 0.8064918108976922 rmse 0.36888515892656853


In [ ]:
rfc = RandomForestClassifier(random_state=0)
CV_rfc = GridSearchCV(estimator=rfc, param_grid=random_grid, cv= 3, scoring = 'f1')
CV_rfc.fit(x_train, y_train)

In [ ]:
CV_rfc.best_params_

## Neutral Network

In [12]:
kappa5 = []
rmse5 = []
f1_5 = []

kf = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
for train_index, test_index in kf.split(x,y):
    x_train, x_test = x[x.index.isin(train_index)], x[x.index.isin(test_index)]
    y_train, y_test = y[y.index.isin(train_index)], y[y.index.isin(test_index)]
    nn = MLPClassifier(random_state = 0)
    nn.fit(x_train, y_train)
    nn_pred = nn.predict(x_test)
    kappa = metrics.cohen_kappa_score(y_test, nn_pred)
    kappa5.append(kappa)
    f1 = metrics.f1_score(y_test, nn_pred)
    f1_5.append(f1)
    rmse = sqrt(metrics.mean_squared_error(y_test, nn_pred))
    rmse5.append(rmse)
print('kappa',np.mean(kappa5), 'f1', np.mean(f1_5), 'rmse', np.mean(rmse5))

/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimizati

kappa 0.6275599114490851 f1 0.7565952581909198 rmse 0.411572123227635


/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
